## Issue Credential

Issuing a credential involves establishing a connection between the issuer and the holder, which is done by following the process outlined in *Example 01 - Connections*. Once the connection is established, the issuer will prepare and send a credential offer, which creates an issue record on both the issuer's and holder's agents. The holder will then retrieve the list of issue records, find the one they wish to accept, and notify the issuer of their acceptance. Finally, the issuer will issue the credential to the holder, completing the process.

In [ ]:
import os
import time
import datetime
import base64
from pprint import pprint
from dotenv import load_dotenv
from termcolor import colored,cprint

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.types import Response
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.models import IssueCredentialRecord, CreateIssueCredentialRecordRequest, CreateIssueCredentialRecordRequestClaims, IssueCredentialRecordCollection, IssueCredentialRecordAllOfProtocolState
from prism_agent_open_api_specification_client.api.issue_credentials_protocol import get_credential_record, get_credential_records, create_credential_offer,accept_credential_offer,issue_credential
from prism_agent_open_api_specification_client.models import CreateManagedDidRequestDocumentTemplatePublicKeysInnerPurpose, CreateManagedDidRequestDocumentTemplatePublicKeysInner
from prism_agent_open_api_specification_client.models import CreateManagedDidRequestDocumentTemplate, CreateManagedDidRequest, CreateManagedDIDResponse
from prism_agent_open_api_specification_client.models import ListManagedDIDResponseInner, ListManagedDIDResponseInnerStatus
from prism_agent_open_api_specification_client.models import DIDDocumentMetadata, DIDOperationResponse, DidOperationSubmission, DIDResponse, DID, Service, ServiceType   
from prism_agent_open_api_specification_client.api.did_registrar import create_managed_did, list_managed_did, publish_managed_did  


### Ultilitary functions

In [ ]:
def get_invitation_str(connection):
    parts = connection.invitation.invitation_url.split("=")
    return parts[1]

def find_credential_record_by_state(client, state):
    credential_records: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=client)

    for offer in credential_records.items:
        if(offer.protocol_state == state):
            return offer
    return None 

def print_credential_record(credential_record):
    print(f"record_id:          {credential_record.record_id}")
    print(f"subject_id:         {credential_record.subject_id}")
    print(f"role:               {credential_record.role}")
    print(f"protocol_state:     {credential_record.protocol_state}")
    print(f"publication_state:  {credential_record.publication_state}")
    print(f"created_at:         {credential_record.created_at}")
    print(f"updated_at:         {credential_record.updated_at}")
    
    
def print_connection(connection):
    print(f"connection_id: {connection.connection_id}")
    print(f"state:         {connection.state}")
    print(f"label:         {connection.label}")
    print(f"my_did:        {connection.my_did}")
    print(f"their_did:     {connection.their_did}")
    print(f"created_at:    {connection.created_at}")

### Client instances

We will create two separate clients, one for the issuer and one for the holder, in order to establish a connection between the two.

note: remember to update the file variables.env with the URLs and API keys provided to you.


In [ ]:
load_dotenv("../BetaProgram/variables.env")
issuerApiKey = os.getenv('ISSUER_APIKEY')
issuerUrl = os.getenv('ISSUER_URL')

holderApiKey = os.getenv('HOLDER_APIKEY')
holderUrl = os.getenv('HOLDER_URL')

issuer_client = Client(base_url=issuerUrl, headers={"apiKey": issuerApiKey})
holder_client = Client(base_url=holderUrl, headers={"apiKey": holderApiKey})

### Create connection

For details on this see "Example 01 - Connections"

In [ ]:
print("Please wait...")

conn_request = CreateConnectionRequest()
conn_request.label = f'Issue credential {datetime.date.today().strftime("%Y-%m-%d %H:%M:%S")}'
issuer_connection: Response[Connection] =  create_connection.sync(client=issuer_client,json_body=conn_request)

invitation = get_invitation_str(issuer_connection)

accept_conn_request = AcceptConnectionInvitationRequest(invitation)
holder_connection: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client,json_body=accept_conn_request)


issuer_connection: Response[Connection] = get_connection.sync(client=issuer_client,connection_id=issuer_connection.connection_id)
holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)

while (issuer_connection.state != 'ConnectionResponseSent' and holder_connection.state != 'ConnectionResponseReceived'):
    issuer_connection: Response[Connection] = get_connection.sync(client=issuer_client,connection_id=issuer_connection.connection_id)
    holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)
    print("Issuer State: {} / Holder State: {} \n".format(issuer_connection.state,holder_connection.state))
    time.sleep(1)
    
print("Connection established between Issuer and Holder!")
print("\nIssuer connection:\n")
print_connection(issuer_connection)
print("\nHolder connection:\n")
print_connection(holder_connection)

### Holder - Create PRISM DID (TBD)


In [ ]:
public_key = CreateManagedDidRequestDocumentTemplatePublicKeysInner(id="key1", purpose=CreateManagedDidRequestDocumentTemplatePublicKeysInnerPurpose.AUTHENTICATION)
service = Service(id="service1", type=ServiceType.MEDIATORSERVICE, service_endpoint=[f"{holderUrl}/didcomm"])
document_template = CreateManagedDidRequestDocumentTemplate([public_key],[])
did_request = CreateManagedDidRequest(document_template)

did_subject: Response[CreateManagedDIDResponse] = create_managed_did.sync(client=holder_client, json_body=did_request)

pprint(f"{did_subject.long_form_did}")

### Issuer - Create credential claim

The issuer creates the credential claim object to store the attributes that will be part of the verifiable credential.

In [ ]:
data = {
        "firstname": 'Joe',
        "lastname": 'Soap',
        "birthdate": '11/11/1999'
      }

credential_claims = CreateIssueCredentialRecordRequestClaims().from_dict(data)

### Issuer - Send credential offer

The issuer creates a `CreateIssueCredentialRecordRequest`. It contains the `subject_id`, the `claims` and other metadata. This object is passed to the `create_credential_offer` endpoint.
The `create_credential_offer` call creates an `IssueCredentialRecord` on the issuer side, it also sends the credential offer to the holder.

In [ ]:
credential_offer = CreateIssueCredentialRecordRequest(subject_id=issuer_connection.their_did, claims=credential_claims, schema_id="1234", validity_period=3600, automatic_issuance=False, await_confirmation=False)

issuer_credential_offer: Response[IssueCredentialRecord] = create_credential_offer.sync(client=issuer_client,json_body=credential_offer)

print_credential_record(issuer_credential_offer)

### Holder - Wait for credential offer

The holder waits to receive the credential offer. When received, it will show up in the holder's credential records list as a new entry with `protocol_state` equal to `OfferReceived`. The code below waits until a credential offer is received and takes the corresponding credential_record.

In [ ]:
holder_credential_record = find_credential_record_by_state(holder_client, IssueCredentialRecordAllOfProtocolState.OFFERRECEIVED)

while(holder_credential_record == None):
    holder_credential_record = find_credential_record_by_state(holder_client, IssueCredentialRecordAllOfProtocolState.OFFERRECEIVED)
    time.sleep(1)
    
print_credential_record(holder_credential_record)

### Holder - Accept credential offer

The holder uses the `accept_credential_offer` endpoint to accept the credential offer. It must provide the `record_id` of the offer. Accepting the credential offer tells the Issuer that the credential can be issued.

In [ ]:
holder_credential_record: Response[IssueCredentialRecord] = accept_credential_offer.sync(client=holder_client, record_id=holder_credential_record.record_id)
print_credential_record(holder_credential_record)

### Issuer - Wait for credential request

The issuer waits to receive the credential request. When received, the credential record state will change to `RequestReceived` in the issuer's credential records list. The code below waits until a credential request is received and takes the corresponding credential_record.

In [ ]:
issuer_credential_offer = get_credential_record.sync(client=issuer_client, record_id=issuer_credential_offer.record_id)

while(issuer_credential_offer.protocol_state != IssueCredentialRecordAllOfProtocolState.REQUESTRECEIVED):
    issuer_credential_offer = get_credential_record.sync(client=issuer_client, record_id=issuer_credential_offer.record_id)
    print("protocol_state: {}\n".format(issuer_credential_offer.protocol_state))
    time.sleep(1)
    
print_credential_record(issuer_credential_offer)

### Issuer - Accept credential request (issue credential to holder)

The issuer uses the `issue_credential` endpoint to issue the credential. It must provide the `record_id`. When issued the record state changes to `CredentialSent` and the credential is send to the holder.

In [ ]:
issuer_credential_offer: Response[IssueCredentialRecord] = issue_credential.sync(client=issuer_client, record_id=issuer_credential_offer.record_id)
print_credential_record(issuer_credential_offer)

### Holder - Wait for credential

The holder waits to receive the credential. When received, it will be added to the holder's credential record and the `protocol_state` will be updated to `CredentialReceived`. The code below waits until a credential is received.

In [ ]:
holder_credential_record = get_credential_record.sync(client=holder_client, record_id=holder_credential_record.record_id)

while(holder_credential_record.protocol_state != IssueCredentialRecordAllOfProtocolState.CREDENTIALRECEIVED):
    holder_credential_record = get_credential_record.sync(client=holder_client, record_id=holder_credential_record.record_id)
    print(f"protocol state: {holder_credential_record.protocol_state}")
    time.sleep(1)
    
print_credential_record(holder_credential_record)

### JWT Credential

The JWT Credential is available in the holder credential record. The website https://jwt.io/ can be used to decode the credential.

In [ ]:
print(base64.b64decode(holder_credential_record.jwt_credential).decode())